# 実験概要
## CMNによる特徴量抽出
**ケプストラム平均正規化** (Cepstrum Mean Normalization : CMN) を使用し、特徴量を抽出する。
切り出した波形に対して**離散フーリエ変換** (Discrete Fourier Transform : DFT) を行い、絶対値を取ることで**振幅スペクトル**を得る。
これに対して対数を取って**対数振幅スペクトル**に変換し、逆離散フーリエ変換 (Inverse Discrete Fourier Transform : IDFT) を行うことで、ケプストラム領域へと変換し、先頭50要素を切り出し、結合した100要素の配列を前処理として返す。
この前処理は
```
left, right, posture = pp.slicer("raw\\" + tester.**.**.value)
cepstrum = pp.cmn_denoise(left, right)
```
によって行われる。

## DNNによる寝姿勢分類
DNNによって前処理したデータから学習・分類を行う。

# モジュールのインポート

In [1]:
import numpy as np
import glob

# 自作モジュール
import datapath as dpath
import preprocess as pp

# DataLoaderの定義

In [2]:
# tester以外のrawを読み込む
for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
    print(p)

In [3]:
e = dpath.LMH.H002.value.fl_center
print(e.value)

type, tester, mattress = dpath.getattributes(e, position=False)
print(f"type : {type},  tester : {tester},  mattress : {mattress}")
train_paths = eval(f"dpath.{type}.serch('{mattress}')")
print(train_paths)

raw\LMH\H002\fl_center
type : LMH,  tester : H002,  mattress : fl
[WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M002/fld_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M003/fld_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/M004/fls_left'), WindowsPath('raw/LMH/M004/fls_right'), WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M00

In [4]:
import torch
from typing import Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 標準化 (平均 0 , 分散 1 )
def standardization(a, axis=None, ddof=0):
    a_mean = a.mean(axis=axis, keepdims=True)
    a_std = a.std(axis=axis, keepdims=True, ddof=ddof)
    return (a - a_mean) / a_std

# 学習用データセットを読み込むためのDataLoaderを定義
class train_datasets(Dataset):
    def __init__(self, identity, transforms=None):
        self.transform = transforms
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print(self.type, self.tester, self.mattress)
        print("train")

        # テスト以外のrawを読み込む
        self.train_cepstrum = np.empty((0, 100))  # 100要素の配列
        self.train_posture = np.empty(0)  # 姿勢データ配列

        # mattressに該当するrawのpathを読み込む
        train_paths = eval(f"dpath.{self.type}.serch('{self.mattress}')")
        for p in train_paths:
            if identity.value == p:
                continue
            left, right, posture = pp.slicer(p)
            cepstrum = pp.cmn_denoise(left, right)
            for cep in cepstrum:
                cep = standardization(cep)
                self.train_cepstrum = np.vstack((self.train_cepstrum, cep)) if self.train_cepstrum.size else cep
            self.train_posture = np.append(self.train_posture, posture) if self.train_posture.size else posture

    def __len__(self):
        return len(self.train_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.train_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.train_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture


# test用データセット
class test_datasets(Dataset):
    def __init__(self, identity, transform=None):
        self.transform = transform
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print("test")

        # 前処理したrawを読み込む
        self.left, self.right, self.test_posture = pp.slicer(identity.value)
        self.test_cepstrum = pp.cmn_denoise(self.left, self.right)
        self.test_cepstrum = standardization(self.test_cepstrum)

    def __len__(self):
        return len(self.test_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.test_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.test_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture

# 学習＆検証

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# 被験者
identity = dpath.LMH.M002.value.ka_center

train_val = train_datasets(identity)
test = test_datasets(identity)
batch_size = 128

n_samples = len(train_val)
train_size = int(n_samples * 0.8)    # [train : val] を [8 : 2] に分割
val_size = n_samples - train_size

train, val = torch.utils.data.random_split(train_val, [train_size, val_size])

train_loader = DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

test_loader = DataLoader(
    test,
    shuffle=False,
)

LMH M002 ka
train
data[168 / 223]
data[214 / 268]
data[154 / 223]
data[204 / 215]
data[218 / 238]
data[193 / 215]
data[206 / 223]
data[182 / 208]
data[200 / 208]
data[217 / 283]
data[192 / 223]
data[209 / 253]
data[171 / 356]
data[204 / 310]
data[204 / 301]
data[207 / 345]
data[218 / 321]
data[208 / 370]
data[243 / 578]
data[230 / 399]
data[125 / 306]
data[109 / 244]
data[126 / 313]
data[168 / 223]
data[214 / 268]
data[154 / 223]
data[204 / 215]
data[218 / 238]
data[193 / 215]
data[206 / 223]
data[182 / 208]
data[200 / 208]
data[217 / 283]
data[192 / 223]
data[209 / 253]
data[171 / 356]
data[204 / 310]
data[204 / 301]
data[207 / 345]
data[218 / 321]
data[208 / 370]
data[243 / 578]
data[230 / 399]
data[125 / 306]
data[109 / 244]
data[126 / 313]
test
data[277 / 594]


## 学習

In [7]:
import torch.optim as optim
import torch.nn as nn
import model_resnet as resnet
import model_har as har

# モデルのインスタンス化
net = har.HAR(num_classes=4).to(device)

# 誤差関数を交差エントロピーで計算
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.SGD(net.parameters(), lr = 1e-3, momentum=0.9)

# 学習
n_epoch = 100
for epoch in range(n_epoch):
    # 精度と損失の初期化
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_val = 0, 0

    # 学習
    for train_input, train_label in train_loader:
        n_train += len(train_label)

        # 入力と正解ラベルをGPU上に移動
        input = train_input.to(device)
        label = train_label.to(device)
        # print(f'input : {input.shape}, label : {label.shape}')

        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        predicted = torch.max(output, 1)[1]

        train_loss += loss.item()
        train_acc += (predicted == label).sum().item()

    # 検証
    for val_input, val_label in val_loader:
        n_val += len(val_label)

        val_input = val_input.to(device)
        val_label = val_label.to(device)

        val_output = net(val_input)
        val_loss = criterion(val_output, val_label)

        val_predicted = torch.max(val_output, 1)[1]

        val_loss += val_loss.item()
        val_acc += (val_predicted == val_label).sum().item()

    # 精度を確率に変換
    train_acc /= n_train
    val_acc /= n_val
    train_loss = train_loss * batch_size / n_train
    val_loss = val_loss * batch_size / n_val

    if not epoch%1:
        print(f"Epoch[{epoch+1}/{n_epoch}] | train_loss: {train_loss:.5f} | train_acc: {train_acc:.5f} | val_loss: {val_loss:.5f} | val_acc: {val_acc:.5f}")


Epoch[1/100] | train_loss: 1.41625 | train_acc: 0.25980 | val_loss: 0.21226 | val_acc: 0.26683
Epoch[2/100] | train_loss: 1.41153 | train_acc: 0.25199 | val_loss: 0.22012 | val_acc: 0.25300
Epoch[3/100] | train_loss: 1.41082 | train_acc: 0.25881 | val_loss: 0.21685 | val_acc: 0.24579
Epoch[4/100] | train_loss: 1.40316 | train_acc: 0.26264 | val_loss: 0.21843 | val_acc: 0.25661
Epoch[5/100] | train_loss: 1.41430 | train_acc: 0.24929 | val_loss: 0.22218 | val_acc: 0.25180
Epoch[6/100] | train_loss: 1.40614 | train_acc: 0.26591 | val_loss: 0.21155 | val_acc: 0.25661
Epoch[7/100] | train_loss: 1.39972 | train_acc: 0.27344 | val_loss: 0.21395 | val_acc: 0.26442
Epoch[8/100] | train_loss: 1.40327 | train_acc: 0.26477 | val_loss: 0.21627 | val_acc: 0.25240
Epoch[9/100] | train_loss: 1.39908 | train_acc: 0.26648 | val_loss: 0.21396 | val_acc: 0.27945
Epoch[10/100] | train_loss: 1.39950 | train_acc: 0.26974 | val_loss: 0.21737 | val_acc: 0.28546
Epoch[11/100] | train_loss: 1.39928 | train_acc: 

# 推定

In [ ]:
test_loss, test_acc = 0, 0
n_test = 0

for test_input, test_label in test_loader:
        n_test += len(test_label)

        test_input = test_input.to(device)
        test_label = test_label.to(device)

        test_output = net(test_input)
        test_loss = criterion(test_output, test_label)

        test_predicted = torch.max(test_output, 1)[1]

        test_loss += test_loss.item()
        test_acc += (test_predicted == test_label).sum().item()
        
# 精度を確率に変換
test_acc /= n_test
test_loss = test_loss * batch_size / n_test

print(f"loss : {test_loss:.5f}, acc : {test_acc:.5f}")

loss : 1.05141, acc : 0.25632
